In [1]:
# 17_search.ipynb

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/app')

In [2]:
from psql_helpers import read_table_async

In [3]:
from psql_models import *

In [4]:
from psql_access import get_meeting_token
from datetime import datetime
from indexing.redis_keys import RedisKeys
from indexing.worker import IndexingWorker
from redis import Redis


In [5]:
from sqlalchemy import update
from psql_helpers import get_session

async def reset_content_indexing():
    async with get_session() as session:
        await session.execute(
            update(Content).values(is_indexed=False)
        )
        await session.commit()

# Execute
#await reset_content_indexing()

In [8]:
df = await read_table_async(Entity)

In [14]:
df[df['name'].str.contains('Montenegro')]

,id,name,type
5431,5432,Mariana Montenegro,speaker


In [16]:
df[df['is_indexed']]

,id,type,text,timestamp,parent_id,is_indexed,last_update


In [15]:
df = await read_table_async(Thread)

In [ ]:
df

,thread_id,user_id,content_id,entity_id,prompt_id,thread_name,messages,timestamp,meta
0,b585c8c1-a0e1-41b5-a055-446871d343b2,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,None,NaN,None,what is vexa?,"[{""role"": ""user"", ""content"": ""what is vexa?"", ...",2025-01-27 17:29:22.620176+00:00,None
1,e9ff173a-7bb3-450e-9f7b-9b68e2bd6052,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,None,NaN,None,what did we say about software??,"[{""role"": ""user"", ""content"": ""what did we say ...",2025-01-27 17:31:05.647719+00:00,None
2,3b93779b-78a3-4a2f-b0c3-01886cf2fda9,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,None,NaN,None,what did we say about software??,"[{""role"": ""user"", ""content"": ""what did we say ...",2025-01-28 10:26:34.287865+00:00,None
3,379c5f00-a4cd-417d-a627-dceabb76c262,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,None,NaN,None,what did we say about software??,"[{""role"": ""user"", ""content"": ""what did we say ...",2025-01-28 10:31:02.214519+00:00,{'content_ids': ['580bb613-00de-4c4a-b048-2c25...
4,dd9af680-2577-4a58-a2bb-7520c10cd34e,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,None,NaN,None,what did we say about software??,"[{""role"": ""user"", ""content"": ""what did we say ...",2025-01-28 10:31:51.345178+00:00,{'content_ids': ['580bb613-00de-4c4a-b048-2c25...
5,a3967c6d-9392-4c1f-bebb-ff8ddae17b10,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,None,NaN,None,what did we say about software??,"[{""role"": ""user"", ""content"": ""what did we say ...",2025-01-28 10:32:17.838608+00:00,{'content_ids': ['a0c3f703-2a5e-4ed0-9561-8071...
6,157fd9da-42dc-43c1-9900-81dcb33039d6,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,None,NaN,None,how many meetings did we have?,"[{""role"": ""user"", ""content"": ""how many meeting...",2025-01-28 10:32:42.835127+00:00,{'content_ids': ['a0c3f703-2a5e-4ed0-9561-8071...
7,ded29b6d-c31f-4798-a804-13971d375644,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,None,NaN,None,how many meetings did we have?,"[{""role"": ""user"", ""content"": ""how many meeting...",2025-01-28 10:33:27.276101+00:00,{'content_ids': ['a0c3f703-2a5e-4ed0-9561-8071...
8,475e6948-a903-4f63-b067-3084bef1c5fb,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,None,NaN,None,how many meetings did we have?,"[{""role"": ""user"", ""content"": ""how many meeting...",2025-01-28 10:33:53.483767+00:00,{'content_ids': ['a0c3f703-2a5e-4ed0-9561-8071...
9,9d977ba9-4c7c-4e01-ab5c-d08215a96b4c,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,None,3484.0,None,who is this guy?,"[{""role"": ""user"", ""content"": ""who is this guy?...",2025-01-28 10:37:52.402732+00:00,None


: 

In [8]:
df['is_indexed'].value_counts()


is_indexed
False    3683
Name: count, dtype: int64

In [9]:
df['type'].value_counts()


type
meeting    3683
Name: count, dtype: int64

In [30]:
processor = ContentProcessor(
            qdrant_engine=self.qdrant_engine,
            es_engine=self.es_engine
        )

NameError: name 'self' is not defined